In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

# Código

```{note}
El proyecto completo está disponible en [github](https://github.com/GuiGel/MedDocAn).
```

## Meddocan pipeline

Una de las tareas es obtener un preprocesso correcto de los informes clínicos a traves de un objeto `spacy.tokens.Doc` a partir de cualquier cadena de caracteres. Por ello utilizaremos el `meddocan.language.pipeline.meddocan_pipeline` creado con la ayuda de la biblioteca [spaCy](https://spacy.io/) para adaptarlo a nuestras necesidades. El código relativo a la creación del pipeline se puede encontrar en el paquete [meddocan/language](https://github.com/GuiGel/MedDocAn/tree/master/meddocan/language).

Por otra parte tenemos que poder leer y extraer la información relevante de los documentos provisto en el formato Brat. Es decir que cada informe clínico esta compuesto de 2 ficheros. Uno contiene el texto bruto y el otro las anotaciones. El código relativo de esa parte se encuentra en el paquete [meddocan/data](https://github.com/GuiGel/MedDocAn/tree/master/meddocan/data).

Para ver cómo funciona, seleccionamos un informe médico gracias al objeto `meddocan.data.docs_iterators.GsDocs` que permite acceder a los documentos del dataset meddocan directamente como objetos `spacy.tokens.Doc` con varios atributos específicos.

In [2]:
from pathlib import Path
from typing import List, Tuple

import pandas as pd
from spacy import displacy

from meddocan.data import meddocan_zip, ArchiveFolder
from meddocan.data.containers import BratAnnotations, BratFilesPair, BratSpan
from meddocan.data.docs_iterators import GsDocs

gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)

El objecto `doc_with_brat_pair` creado por `GsDocs` tiene 2 atributos.

In [3]:
[attr for attr in vars(doc_with_brat_pair).keys()]

['brat_files_pair', 'doc']

El atributo `brat_files_pair` es un objeto `meddocan.data.docs_iterators.BratFilesPair` que indica la ubicación de los ficheros originales correspondiendo al attributo `doc`.

In [4]:
pd.DataFrame(
    [type(doc_with_brat_pair.brat_files_pair).__qualname__,
     doc_with_brat_pair.brat_files_pair.ann.name,
     doc_with_brat_pair.brat_files_pair.txt.name],
    index=["type", "txt", "ann"]
).T

,type,txt,ann
0,BratFilesPair,S0004-06142005000500011-1.ann,S0004-06142005000500011-1.txt


Lo que hace `GsDocs` es crear un objecto `Doc` a partir de un objeto `meddocan.data.docs_iterators.DocWithBratPair` utilizando el `MedocanPipeline`.

En una primera fase el ``MedocanPipeline`` recibe el texto contenido en el fichero original *S0004-06142005000500011-1.txt* como argumento.  

Miramos el contenido del fichero utilizando el atributo `doc`.

In [5]:
gold = doc_with_brat_pair.doc
gold

Datos del paciente.
Nombre:  Ernesto.
Apellidos: Rivera Bueno.
NHC: 368503.
NASS: 26 63514095.
Domicilio:  Calle Miguel Benitez 90.
Localidad/ Provincia: Madrid.
CP: 28016.
Datos asistenciales.
Fecha de nacimiento: 03/03/1946.
País: España.
Edad: 70 años Sexo: H.
Fecha de Ingreso: 12/12/2016.
Médico:  Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Informe clínico del paciente: Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, 

  En un segunda fase se le asigna las entidades extraídas del fichero *S0004-06142005000500011-1.ann* utilizando el método ``spacy.tokens.Doc.set_ents``.   
  Miramos el fichero al formato *.brat* que contiene la anotaciones para hacerse una idea.

In [6]:
print(doc_with_brat_pair.brat_files_pair.ann.read_text())

T1	FECHAS 215 225	03/03/1946
T2	CORREO_ELECTRONICO 2421 2439	nnavcu@hotmail.com
T3	PAIS 2406 2412	España
T4	TERRITORIO 2398 2404	Madrid
T5	TERRITORIO 2392 2397	28036
T6	CALLE 2365 2391	c/ del Abedul 5-7, 2º dcha
T7	NOMBRE_PERSONAL_SANITARIO 303 326	Ignacio Navarro Cuéllar
T8	NOMBRE_PERSONAL_SANITARIO 2341 2364	Ignacio Navarro Cuéllar
T9	EDAD_SUJETO_ASISTENCIA 389 396	70 años
T10	ID_TITULACION_PERSONAL_SANITARIO 334 345	28 28 70973
T11	FECHAS 282 292	12/12/2016
T12	SEXO_SUJETO_ASISTENCIA 261 262	H
T13	EDAD_SUJETO_ASISTENCIA 247 254	70 años
T14	PAIS 233 239	España
T15	TERRITORIO 166 171	28016
T16	TERRITORIO 154 160	Madrid
T17	CALLE 107 130	Calle Miguel Benitez 90
T18	ID_ASEGURAMIENTO 82 93	26 63514095
T19	ID_SUJETO_ASISTENCIA 68 74	368503
T20	NOMBRE_SUJETO_ASISTENCIA 49 61	Rivera Bueno
T21	NOMBRE_SUJETO_ASISTENCIA 29 36	Ernesto



Ahora que tenemos una idea mas clara de nuestros datos originales, observamos la serie de pre-procesos aplicados por el `meddocan_pipeline` y ``GsDocs`` gracias a la instancia `gold` del objeto ``Doc`` para preparar el dataset a fin de entrenar un red neuronal con Flair.

```{note}
En el ejemplo solo miramos las 3 primeras lineas del objeto ``Doc``.
```

In [7]:
max_lines = 3

for i, sent in enumerate(gold.sents):
    print(f"--------------- Sentence {i + 1} ---------------")
    a = zip(*((tok.text, tok.ent_iob_, tok.ent_type_) for tok in sent))
    df = pd.DataFrame(a, index=["text", "bio", "etiqueta"])
    display(df.T)
    if i >= max_lines - 1:
        break

---------------------- Sentence 1 ------------------------------


,text,bio,etiqueta
0,Datos,O,
1,del,O,
2,paciente,O,
3,.,O,
4,\n,O,


---------------------- Sentence 2 ------------------------------


,text,bio,etiqueta
0,Nombre,O,
1,:,O,
2,,O,
3,Ernesto,B,NOMBRE_SUJETO_ASISTENCIA
4,.,O,
5,\n,O,


---------------------- Sentence 3 ------------------------------


,text,bio,etiqueta
0,Apellidos,O,
1,:,O,
2,Rivera,B,NOMBRE_SUJETO_ASISTENCIA
3,Bueno,I,NOMBRE_SUJETO_ASISTENCIA
4,.,O,
5,\n,O,


Para entender un poco mejor lo que hacemos miramos los differentes componentes del `MeddocanPipeline`.

In [8]:
pd.DataFrame(gs_docs.nlp.pipe_names, columns=["componentes"]).T

,0,1,2,3
componentes,missaligned_splitter,line_sentencizer,predictor,write_methods


1. El primer elemento de nuestro pipeline es el tokenizer seguido del componente `missaligned_splitter` que nos permite afinar la tokenización de tal forma que cada token se corresponda exactamente con una etiqueta al formato BIO.
2. El segundo componente, `line_sentencizer` permite partir el texto en frases. En este caso se corresponde a un párrafo.
3. El componente `predictor` nos permite utilizar un modelo de `Flair` de tal forma que se integra al pipeline. De esa mañera se puede hacer directamente predicciones utilizando un objeto `Doc` y un modelo entrenado previamente.
4. El componente `write_methods` añade los métodos ``to_connl03`` y ``to_ann`` al objecto ``Doc`` que sirven a crear los ficheros necesarios para:
    - Crear un `flair.data.Corpus` que va a permitir entrenar un modelo con `Flair`.
    - Evaluar un modelo utilizando el script de evaluación propio de la competición.

``````{note}
Hemos integrado el [script de evaluation](https://github.com/PlanTL-GOB-ES/MEDDOCAN-Evaluation-Script) dentro de nuestra librería con algunas modificaciones y un poco mas de documentación, con el objetivo de unificar el workflow del entrenamiento hasta la evaluación.  
La evaluación se hace entonces directamente desde nuestra librería gracias al commando: 

```console
$ meddocan eval --help
Usage: meddocan eval [OPTIONS] MODEL NAME
Evaluate the model with the `meddocan` metrics.
    
    Compute f1-score for Ner (start, end, tag), Span (start, end) and merged
    span if not there is no number or letter between consecutive span.

    The function produce the following temporary folder hierarchy:

    evaluation_root
    ├── golds
    │   ├── dev
    |   |    └── brat
    |   |       ├── file-1.ann
    |   |       ├── file-1.txt
    |   |       ├── ...
    |   |       └── file-n.ann
    |   └── test
    |        └── brat
    |           ├── file-1.ann
    |           ├── file-1.txt
    |           ├── ...
    |           └── file-n.ann
    │       
    └── name
        ├── dev
        |    └── brat
        |       ├── file-1.ann
        |       ├── file-1.txt
        |       ├── ...
        |       └── file-n.ann
        └── test
             └── brat
                ├── file-1.ann
                ├── file-1.txt
                ├── ...
                └── file-n.ann

    Then the model is evaluate producing the following files:

    evaluation_root/name
    ├── dev
    │   ├── ner
    │   └── spans
    └── test
        ├── ner
        └── spans

    And the temporary folder are removed.

    Args:
        model (str): Path to the ``Flair`` model to evaluate.
        name (str): Name of the folder that will holds the results produced by\
            the ``Flair`` model.
        evaluation_root (str): Path to the root folder where the
            results will be stored.
        sentence_splitting (Path): Path to the sub-directory
            `sentence_splitting`. This directory is mandatory to compute the
            `leak score` evaluation metric.
        force (bool, optional): Force to create again the golds standard files.
            Defaults to False.

Arguments:
  MODEL  Path to the Flair model to evaluate.  [required]
  NAME   Name of the folder that will holds the results produced by the
         ``Flair`` model.  [required]

Options:
  --evaluation-root PATH     Path to the root folder where the results will be
                             stored.
  --sentence-splitting PATH  The sub-directory `sentence_splitting` is
                             mandatory to compute the `leak score` evaluation
                             metric.
  --device TEXT              Device to use.  [default: cuda:0]
  --help                     Show this message and exit.        
```
``````

La entidades anotadas se obtienen utilizando el atributo ``ents`` de nuestro objecto ``spacy.tokens.Doc``.

In [9]:
pd.DataFrame(
    zip(*[(ent.text, ent.start_char, ent.end_char) for ent in gold.ents]),
    index=["Tag", "start", "end"]
).T

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Tag,Ernesto,Rivera Bueno,368503,26 63514095,Calle Miguel Benitez 90,Madrid,28016,03/03/1946,España,70 años,...,12/12/2016,Ignacio Navarro Cuéllar,28 28 70973,70 años,Ignacio Navarro Cuéllar,"c/ del Abedul 5-7, 2º dcha",28036,Madrid,España,nnavcu@hotmail.com
start,29,49,68,82,107,154,166,215,233,247,...,282,303,334,389,2341,2365,2392,2398,2406,2421
end,36,61,74,93,130,160,171,225,239,254,...,292,326,345,396,2364,2391,2397,2404,2412,2439


Nuestras entidades son en este ejemplo compuestas del tag y del la position de la entidad en la cadena de caracteres original.

Ahora si queremos algo mas visual podemos utilizar la function ``spacy.displacy.render()`` que nos permite trabajar con el objecto ``Doc``.

In [10]:
displacy.render(gold, style="ent")

## Entrenar un modelo con Flair

La clase ``GsDocs`` sirve a preparar los datos de forma que puedan ser leídos por Flair.  Esto lo hacemos a través del objeto ``meddocan.data.corpus.MEDDOCAN`` que hereda de ``flair.datasets.ColumnCorpus``.  

Este corpus permite a la biblioteca Flair acceder directamente a los conjuntos de datos de entrenamiento, validación y prueba.

In [11]:
from meddocan.data.corpus import MEDDOCAN

corpus = MEDDOCAN(sentences=True, in_memory=True, document_separator_token="-DOCSTART-")

2022-10-17 11:32:05,675 Reading data from /tmp/tmpzbtses7j
2022-10-17 11:32:05,675 Train: /tmp/tmpzbtses7j/train
2022-10-17 11:32:05,676 Dev: /tmp/tmpzbtses7j/dev
2022-10-17 11:32:05,676 Test: /tmp/tmpzbtses7j/test


El corpus creado podemos utilizar los métodos de los cuales hereda nuestro objeto como el método especial ``__str__`` que escribe en stdout el numero de objetos ``flair.tokens.Sentence`` contenido en cada subconjunto de datos. Es decir cuantos parafos tenemos en total en cada uno de nuetros conjuntos de datos.

In [12]:
print(corpus)

Corpus: 10811 train + 5518 dev + 5405 test sentences


Para entrenar los modelos con la libreria ``Flair`` basta seguir el ejemplo siguiente. 

```{note}
Todos nuestros experimentos se pueden encontrar en la carpeta [experiments](https://github.com/GuiGel/MedDocAn/tree/master/experiments) y siguen este formato.  
La única diferencia es la adición de las librerías hyperopt [^1] y Tensorboard [^2] para obtener una trazabilidad de los entrenamientos mediante el registro de diversos parámetros y resultados.
```

[^1]: https://github.com/hyperopt/hyperopt
[^2]: https://pytorch.org/docs/stable/tensorboard.html

```python
from flair.data import Corpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from meddocan.data.corpus import MEDDOCAN

# 1. Obtener el corpus
corpus: Corpus = MEDDOCAN(
    sentences=True, document_separator_token="-DOCSTART-"
)
print(corpus)

# 2. Que label se quiere predecir?
label_type = 'ner'

# 3. Crear el diccionario de labels a partir del corpus
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

# 4. Inicializar los embeddings generados por el transformador utilizando el contexto
embeddings = TransformerWordEmbeddings(model='dccuchile/bert-base-spanish-wwm-cased',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. Inicializar etiquedator simple (no CRF, no RNN, no reprojección)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. Initializar el trainer
trainer = ModelTrainer(tagger, corpus)

# 7. Ejecutar el fine-tuning
trainer.fine_tune('experiments/meddocan',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  )
```

## Inferencia

Una vez el modelo entrenado, la inferencia se hace gracias al ``meddocan_pipeline`` justamente porque nos permitte integrar un modelo de ``Flair `` gracias a su componente ``predictor`` como lo vamos a ver a continuación. 

```{note}
Aquí utilizamos por el ejemplo un modelo de FLair pre-entreando con los embeddings de Flair y una red LSTM-CRF y entrenado sobre el dataset *CONLL-2002* en inglès.Este dataset consiste en artículos de noticias anotados con las categorías LOC, PER y ORG que son diferentes de las categorías de MEDDOCAN.
```

In [13]:
from meddocan.language.pipeline import meddocan_pipeline

nlp = meddocan_pipeline("flair/ner-english-fast")
sys = nlp(gold.text)

2022-10-17 11:32:13,228 loading file /home/wave/.meddocan/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2022-10-17 11:32:14,922 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


El objeto sys es un objeto ``spacy.tokens.Doc`` al igual de el objeto ``gold``.  

La única diferencia entre sys y gold es que sys contiene entidades que le son asignadas por un modelo entrenado con los algoritmos de Flair, mientras que en el caso de gold provienen de la lectura de un archivo *.ann*.

Entonces para visualizar las predicciones de nuestro model, lo tenemos igual de fácil que antes. Basta utilizar la function ``spacy.displacy.render()``.

In [14]:
displacy.render(sys, style="ent")

```{note}
Vemos a ojo que el modelo de Flair `flair/ner-english-fast´ parece detectar correctamente el span de ciertas endidades que el etiqueta como ``LOC`` o ``PERS`` y que effectivamente son direcciónes o personas. 
```

## Evaluation

La evaluación originalmente provistas a traves del [script de evaluation](https://github.com/PlanTL-GOB-ES/MEDDOCAN-Evaluation-Script) que re-utilizamos, utiliza el texto original asi como su anotación al formato *brat* para calcular las métricas segun las tareas ``Subtrack1``, ``Subtrack2 [Strict]`` y ``SubTrack2 [Merged]``.

Para evaluar nuestros modelos, utilizamos el texto original a partir del cual se ha creado el documento sys asi como su atributo ``_.to_ann``. Ese método permite codificar el atributo ``ents`` del objeto sys en un fichero siguiendo el formado brat como se puede ver a continuación.

In [15]:
from tempfile import TemporaryDirectory

with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    sys._.to_ann(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)

T_0	ORG 0 18	Datos del paciente
T_1	PER 29 36	Ernesto
T_2	PER 49 61	Rivera Bueno
T_3	ORG 76 80	NASS
T_4	PER 107 127	Calle Miguel Benitez
T_5	LOC 132 141	Localidad
T_6	LOC 143 152	Provincia
T_7	LOC 154 160	Madrid
T_8	ORG 162 164	CP
T_9	MISC 194 213	Fecha de nacimiento
T_10	ORG 233 239	España
T_11	ORG 241 245	Edad
T_12	ORG 264 280	Fecha de Ingreso
T_13	ORG 294 300	Médico
T_14	ORG 303 332	Ignacio Navarro Cuéllar NºCol
T_15	MISC 377 404	Paciente de 70 años de edad
T_16	MISC 1758 1767	En la UIV
T_17	PER 2341 2364	Ignacio Navarro Cuéllar
T_18	PER 2368 2378	del Abedul
T_19	LOC 2398 2404	Madrid
T_20	ORG 2406 2414	España E



Para hacer esto mas automatizado, al igual que la clase ``from meddocan.data.docs_iterators.GsDocs``, tenemos la clase ``from meddocan.data.docs_iterators.SysDocs`` que utiliza un modelo de Flair para detectar entidades sobre los documentos de cada sub-conjunto de datos. Veamos un ejemplo:

In [16]:
from meddocan.data.docs_iterators import SysDocs
from meddocan.data import ArchiveFolder

import torch
import flair

flair.device = torch.device("cuda:1")

sys_docs = iter(SysDocs(archive_name=ArchiveFolder.test, model="flair/ner-spanish-large"))

2022-10-17 11:32:17,742 loading file /home/wave/.meddocan/models/ner-spanish-large/045ad6c7dc21e0eb85935dce0544eec65f8c63c58412154df4dee7ff5f11665b.d4d3456316d2951bc100d060bd63a690b33af6d273adffa1b90df32328ed3257
2022-10-17 11:32:31,829 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


Gracias a las classe ``GsDocs`` y ``SysDocs`` podemos producir facilmente los ficheros requeridos para usar el script de evaluación provisto a traves de la la linea de comando *meddocan eval*.

Como curiosidad podemos explicar como se pueden calcular las metricas.

In [17]:
gs_docs = iter(GsDocs(archive_name=ArchiveFolder.test))

Recuperamos los ``Iterator`` sys_doc y gold_doc y comprobamos que corresponden a los mismos documentos originales, tanto en su origen como en su contenido.

In [18]:
sys_doc, gold_doc = next(sys_docs), next(gs_docs)

assert sys_doc.brat_files_pair.ann.name == gold_doc.brat_files_pair.ann.name
assert sys_doc.brat_files_pair.txt.name == gold_doc.brat_files_pair.txt.name
assert sys_doc.doc.text == gold_doc.doc.text

Ahora recuperamos las entidades originales en ``gold_labels`` y las predicciones ``sys_labels``.

In [19]:
from meddocan.evaluation.classes import Ner, Span

gold_labels = set(Ner(ent.start, ent.end, ent.label_) for ent in gold.ents)
sys_labels = set(Ner(ent.start, ent.end, ent.label_) for ent in sys.ents)

Por fin calculamos el score $F_1$ para el documento asi como el recall y la precision.

In [20]:
from typing import TypeVar

T = TypeVar("T", Ner, Span)

def f1(gold_label: List[T], sys_label: List[T]) -> float:
    tp = gold_label.intersection(sys_label)
    fp = sys_label - gold_label
    fn = gold_label - sys_label
    try:
        recall = len(tp) / float(len(tp) + len(fp))
    except ZeroDivisionError:
        recall = 0.0
    try:
        precision = len(tp) / float(len(tp) + len(fn))
    except ZeroDivisionError:
        precision = 0.0
    try:
        f1 = 2 * (recall * precision) / (recall + precision)
    except ZeroDivisionError:
        f1 = 0.0
    return f1, recall, precision

pd.DataFrame(
    f1(gold_labels, sys_labels),
    index=["f1", "recall", "precisión"],
    columns=["Subtrack1"]
).T

,f1,recall,precisión
Subtrack1,0.0,0.0,0.0


El score f1 esta nulo simplemente porque el modelo utilizado no predice las mismas entidades y ademas esta entrenado sobre un dataset en inglès! Pero si se trata unicamente de anonimizar y que usamos solo los ``Span`` puede que no sea lo mismo dado que tanbien tiene que detectar personas o entidas sin etiqueta.

In [21]:
gold_spans = set(Span(ent.start, ent.end) for ent in gold.ents)
sys_spans = set(Span(ent.start, ent.end) for ent in sys.ents)
pd.DataFrame(
    f1(gold_spans, sys_spans),
    index=["f1", "recall", "precision"],
    columns=["Subtrack2[strict]"]
).T

,f1,recall,precision
Subtrack2[strict],0.285714,0.285714,0.285714


De hecho, vemos que incluso con un modelo entrenado en un conjunto de datos muy diferente con tan sólo 4 entidades, conseguimos anonimizar un poco mas de un quarto de los span desados.